# SQL

Проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

### Описание данных

<img src=https://joxi.ru/vAW7MYKIO7YQdA.jpg></div> <br>

### Задания

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Исследуем таблицы — выведим первые строки**

In [2]:
for i in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Таблица ', i)
    display(pd.io.sql.read_sql(f'''SELECT * FROM {i} LIMIT 5''', con = engine))
    print()
    print()

Таблица  books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




Таблица  authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd




Таблица  publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company




Таблица  ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




Таблица  reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Посчитаем, сколько книг вышло после 1 января 2000 года.**

In [3]:
query = '''
SELECT
    COUNT(book_id)
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''

In [4]:
pd.io.sql.read_sql(query, con = engine)

,count
0,819


<font color="blue">После 1 января 2000 года вышло 819 книг.</font>

**Для каждой книги посчитаем количество обзоров и среднюю оценку.**

In [5]:
query1 = '''
SELECT
    books.book_id,
    books.title,
    COUNT(DISTINCT review_id) as count_reviews,
    AVG(rating) as avg_rating
FROM 
    books
FULL JOIN reviews ON  reviews.book_id = books.book_id
FULL JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    count_reviews DESC,
    avg_rating DESC
LIMIT 10
'''

In [6]:
query2 = '''
WITH a AS (
SELECT
    books.book_id,
    books.title,
    COUNT(reviews.review_id) as count_reviews
FROM
    books
FULL JOIN reviews ON  reviews.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    count_reviews DESC
    ),
b as (
SELECT
    books.book_id,
    books.title,
    AVG(ratings.rating) as avg_rating
FROM
    books
FULL JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    avg_rating DESC
    )
SELECT
    a.book_id,
    a.title,
    a.count_reviews,
    b.avg_rating
FROM
    a
FULL JOIN b ON  a.book_id = b.book_id
ORDER BY
    count_reviews DESC,
    avg_rating DESC
LIMIT 10
'''

In [7]:
pd.io.sql.read_sql(query1, con = engine)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


In [8]:
pd.io.sql.read_sql(query2, con = engine)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


<font color="blue">Больше всего обзоров (7) на книгу Twilight (Twilight #1) (Сумерки).</font>

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.**

In [9]:
query = '''
SELECT
    publisher,
    COUNT(book_id) as count_book
FROM
    publishers
FULL JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    count_book DESC
LIMIT 10
'''

In [10]:
pd.io.sql.read_sql(query, con = engine)

,publisher,count_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


<font color="blue">Наибольшее число книг выпустило издательство Penguin Books.</font>

**Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.**

In [11]:
query = '''
WITH data as (
SELECT
   a.author,
   r.book_id,
   COUNT(rating) as count_rating,
   AVG(rating) as avg_rating
FROM
   ratings r
LEFT JOIN books b ON r.book_id = b.book_id
LEFT JOIN authors a ON b.author_id = a.author_id
GROUP BY
    a.author,
    r.book_id
HAVING
   COUNT(rating) > 50
   )

SELECT
    author,
    AVG(avg_rating) as avg_rating
FROM
    data
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''

In [12]:
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


<font color="blue">Cамая высокая средняя оценка книг у автора J.K. Rowling/Mary GrandPré.</font>

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [13]:
query = '''
WITH a AS (
SELECT
    COUNT(review_id) as count_review
FROM 
    reviews
WHERE username IN
    (SELECT
         username
     FROM 
         ratings
     GROUP BY
         username
     HAVING
         COUNT(ratings) > 50)
GROUP BY 
   username
   )

SELECT
    ROUND(AVG(count_review))
FROM
    a
'''

In [14]:
pd.io.sql.read_sql(query, con = engine)

,round
0,24.0


# Выводы

По итогам запросов для данных из предоставленной БД:

- После 1 января 2000 года было выпущенно 819 книг.  
- Книга "Сумерки" насчитывает больше всего пользовательских обзоров (1120) и имеет среднюю оценку в 3.7 балла.  
- Издательство Penguin Books выпустило больше всех книг (42 книги).  
- Автор с самой высокой средней оценкой книг (4.3) - Джоан Роулинг.  
- В среднем, пользователь который поставил больше 50 оценок, пишет 24 обзора.